# Bayesian Optimization with Ax

### Setup

#### Import Libraries

This script uses the Ax Bayesian Optimization model to try to find the best parameters for optimizing a battery for the University of Utah Chemical Engineering Car Club. There is a 2D example of this script in the AxDemonstration notebook. 

In [3]:
from ax.service.ax_client import AxClient
import pandas as pd
from ax.service.utils.instantiation import ObjectiveProperties
from IPython.display import clear_output
from time import sleep

### Train the Model 

Using the collected data, train the Ax model and use it to suggest a new set of parameters to test. 

The output of this code will be a set of the parameters that you need to test next.

Run the code below to get a list of parameters to try. 

In [16]:
# Read the CSV file
df = pd.read_csv('../Datasets/ObservedData.csv')

ax_client = AxClient(verbose_logging=False)

# Define the parameters based on your CSV columns
parameters = [
    {"name": "SizeOfCell", "type": "choice", "values": [1.53, 1.8, 2.0]},
    {"name": "Magnesium", "type": "range", "bounds": [0.0, 1.0]},
    {"name": "LayersOfPaper", "type": "choice", "values": [1, 2]},
    {"name": "Chloride", "type": "range", "bounds": [0.9, 1.5]}
]

# Create the experiment
ax_client.create_experiment(
    name="battery_optimization",
    parameters=parameters,
    objectives={"y": ObjectiveProperties(minimize=False)}
)

# Attach existing trials from the CSV
for i, row in df.iterrows():
    parameters = {"SizeOfCell": row["Size of the Cell"], "Magnesium": row["Magnesium Dioxide wt%"], "LayersOfPaper": int(row["Layers of Filter Paper"]), "Chloride": row["3M Ammonium Chloride Saturation (mL)"]}
    objective_value = row['Power (W)']
    ax_client.attach_trial(parameters)
    ax_client.complete_trial(trial_index=i, raw_data={"y": objective_value})

# Get next set of parameters to try
parameters, trial_index = ax_client.get_next_trial()

# Compute x3 based on the generated x2
parameters["Graphite"] = 1 - parameters["Magnesium"]

# Assume the total weight is 15 grams
total_weight = 15.0  # in grams

# Calculate weights of Magnesium and Graphite based on their wt% of the total weight
magnesium_weight = parameters["Magnesium"] * total_weight
graphite_weight = parameters["Graphite"] * total_weight

clear_output()
sleep(0.2)


# Print the next set of parameters to try, including weights in grams
print("Next set of parameters to try:")
print(f"Size of the Cell: {parameters['SizeOfCell']:.3f}")
print(f"Magnesium Dioxide: {magnesium_weight:.2f} g ({parameters['Magnesium']*100:.1f}%)")
print(f"Graphite: {graphite_weight:.2f} g ({parameters['Graphite']*100:.1f}%)")
print(f"Layers of Filter Paper: {parameters['LayersOfPaper']} layers")
print(f"3M Ammonium Chloride: {parameters['Chloride']:.3f} ml")

Next set of parameters to try:
Size of the Cell: 1.530
Magnesium Dioxide: 11.67 g (77.8%)
Graphite: 3.33 g (22.2%)
Layers of Filter Paper: 1 layers
3M Ammonium Chloride: 1.246 ml


### Log the Data

Run the code below to get the input prompt to log the data. This number is the average voltage recorded from the battery using the suggested parameters. 

In [ ]:
# Here you would typically run your experiment with these parameters
# and get a result. For this example, let's just use a dummy result
result = float(input("Input the tested variable"))  # Replace this with your actual experimental result

# Complete the trial with the result
ax_client.complete_trial(trial_index=trial_index, raw_data={"y": result})

UnsupportedError: Trial 4 has already been completed with data. To add more data to it (for example, for a different metric), use `Trial.update_trial_data()`.

### Save the Data

Run the cell below to save the new parameters and observed value to the csv file. This will allow the model to incorperate the new data into its future fittings.

In [ ]:
# round parameters to 3 decimal places
parameters = {key: round(value, 3) for key, value in parameters.items()}

# Prepare the new row as a DataFrame
new_row = pd.DataFrame([{
    "Size of the Cell": parameters['SizeOfCell'],
    "Magnesium Dioxide wt%": parameters['Magnesium'],
    "Graphite wt%": parameters['Graphite'],
    "Layers of Filter Paper": parameters['LayersOfPaper'],
    "3M Ammonium Chloride (mL)": parameters['Chloride'],
    "Voltage (V)": result
}])

# Concatenate the new row to the existing DataFrame
df = pd.concat([df, new_row], ignore_index=True)

# Save the updated dataframe to a CSV file
df.to_csv('../Datasets/ObservedData.csv', index=False)

### Print the top parameters found so far

In [ ]:
# Get the best parameters
best_parameters, metrics = ax_client.get_best_parameters()

# Print the best parameters to 3 decimal places
print("\nBest parameters:")
for key, value in best_parameters.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

print("\nBest target value:")
print(metrics[0]['y'])


Best parameters:
SizeOfCell: 1.607
Magnesium: 0.281
LayersOfPaper: 2
Chloride: 8.000

Best target value:
380.0
